# Classical results
---
In this notebook, we obtain states of spin networks by contractions of tensors, using package ```tensornetwork```. Results of this method are used to verify fidelity of states obtained using qunatum methods.

All names used in this notebook are the same as in the  article and all references are refrences to the article.

In [1]:
import tensornetwork as tn
import numpy as np

We define tensor representing node $\iota_k^{m_1m_2m_3m_4}$, which, in our case of spins $\frac{1}{2}$, has shape $2\times2\times2\times2\times2$.

In [2]:
iota = np.array([[[[[ 0.        ,  0.        ],
                    [ 0.        ,  0.        ]],
                   
                   [[ 0.        ,  0.5       ],
                    [-0.5       ,  0.        ]]],

                  [[[ 0.        , -0.5       ],
                    [ 0.5       ,  0.        ]],

                   [[ 0.        ,  0.        ],
                    [ 0.        ,  0.        ]]]],

                 [[[[ 0.        ,  0.        ],
                    [ 0.        ,  0.57735027]],

                   [[ 0.        , -0.28867513],
                    [-0.28867513,  0.        ]]],

                  [[[ 0.        , -0.28867513],
                    [-0.28867513,  0.        ]],

                   [[ 0.57735027,  0.        ],
                    [ 0.        ,  0.        ]]]]])

We define also tensor representing state of link $\alpha_{m_1m_2}$. In our case, singlet state $\frac{1}{\sqrt{2}}\left(|01\rangle-|10\rangle\right)$.

In [3]:
link = np.array([0,1/np.sqrt(2),-1/np.sqrt(2),0]).reshape((2,2))

In [4]:
def norm(state):
    '''Return normalized state'''
    state_conj = tn.replicate_nodes([state], conjugate=True)[0]
    for i in range(len(state.edges)):
        state.edges[i]^state_conj.edges[i]
    n = tn.contractors.auto([state, state_conj]).tensor
    state.tensor = state.tensor/np.sqrt(n)
    return state

# Open node

Construction of node with four free links

$$\left(\Gamma_2'\right)_{km_1'm_2'm_3'm_4'}=\iota_k^{m_1m_2m_3m_4}\alpha_{m_1m_1'}\alpha_{m_2m_2'}\alpha_{m_3m_3'}\alpha_{m_4m_4'}$$

In [7]:
node = tn.Node(iota)
links = [tn.Node(link) for i in range(4)]
node[1]^links[0][0]
node[2]^links[1][0]
node[3]^links[2][0]
node[4]^links[3][0]
state = tn.contractors.auto([node]+links, output_edge_order=[node[0]]+[l[1] for l in links])
state_open_node = norm(state)
state_open_node.tensor.reshape(2**5)

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.35355339, -0.35355339,  0.        ,  0.        , -0.35355339,
        0.35355339,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.40824829,
        0.        , -0.20412414, -0.20412414,  0.        ,  0.        ,
       -0.20412414, -0.20412414,  0.        ,  0.40824829,  0.        ,
        0.        ,  0.        ])

# Pentagram from open node

Construction of pentagram using open node and six links:

$$
\left(\Gamma_5\right)_{kk_1k_2k_3k_4}=
\left(\Gamma_2'\right)_{km_1n_1s_1t_1}

\iota_{k_1}^{m_1m_2m_3m_4}
\iota_{k_2}^{n_1n_2n_3n_4}
\iota_{k_3}^{s_1s_2s_3s_4}
\iota_{k_4}^{t_1t_2t_3t_4}
              
\alpha_{m_2n_2}
\alpha_{s_4t_4}
\alpha_{m_4t_2}
\alpha_{n_3s_3}
\alpha_{m_3s_2}
\alpha_{n_4t_3}
$$

In [8]:
links = [tn.Node(link) for i in range(6)]

n=[]
for i in range(4):
    n.append(tn.Node(iota, name='N'+str(i)))

state_open_node[1]^n[0][1]
state_open_node[2]^n[1][1]
state_open_node[3]^n[2][1]
state_open_node[4]^n[3][1]

links[0][0]^n[0][2]
links[0][1]^n[1][2]

links[4][0]^n[0][3]
links[4][1]^n[2][2]

links[2][0]^n[0][4]
links[2][1]^n[3][2]

links[3][0]^n[1][3]
links[3][1]^n[2][3]

links[5][0]^n[1][4]
links[5][1]^n[3][3]

links[1][0]^n[2][4]
links[1][1]^n[3][4]

state = tn.contractors.auto([state_open_node]+n+links, output_edge_order=[state_open_node[0],n[0][0],n[1][0],n[2][0],n[3][0]])
state_pentagram = norm(state)
state_pentagram.tensor.reshape(2**5)

array([ 4.00891867e-01,  0.00000000e+00,  0.00000000e+00,  4.00891864e-01,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -2.31455024e-01, -2.31455024e-01,  2.67261240e-01,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -2.31455024e-01, -2.31455024e-01, -2.67261240e-01,
        4.00891864e-01,  0.00000000e+00,  0.00000000e+00, -1.33630620e-01,
        0.00000000e+00, -2.67261240e-01,  2.67261240e-01, -1.13002749e-18])

# Open pentagram

In [28]:
N = 5
pentagram_links = [[0,4],[1,8],[2,12],[3,16],[5,9],[6,13],[7,17],[10,14],[11,18],[15,19]]
link_states = [np.array([0,1/np.sqrt(2),-1/np.sqrt(2),0]).reshape((2,2)) for i in range(2*N)]

n=[]
for i in range(4):
    n.append(tn.Node(iota, name='N'+str(i)))
    
l=[]
for link in link_states:
    l.append(tn.Node(link, name='L'+str(i)))
    
for i,(p,q) in enumerate(pentagram_links):
    n[p//4][p%4+1]^l[i][0]
    if (q//4)<4:
        l[i][1]^n[q//4][q%4+1]

    
state_open_pentagram = tn.contractors.auto(n+l, output_edge_order=[nn.edges[0] for nn in n]+[l[3].edges[1],l[6].edges[1],l[8].edges[1],l[9].edges[1]])
state_open_pentagram = norm(state_open_pentagram)

In [29]:
state_open_pentagram.tensor.reshape(2**8)

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  2.00445933e-01, -2.00445933e-01,  0.00000000e+00,
        0.00000000e+00, -2.00445933e-01,  2.00445933e-01,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  2.31455027e-01,
        0.00000000e+00, -1.15727512e-01, -1.15727512e-01,  0.00000000e+00,
        0.00000000e+00, -1.15727512e-01, -1.15727512e-01,  0.00000000e+00,
        2.31455027e-01,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -5.10948616e-18, -5.10948616e-18,  0.00000000e+00,
        0.00000000e+00,  5.10948616e-18,  5.10948616e-18,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -4.72819157e-18,
        0.00000000e+00, -

# Dekagram

In [30]:
state_open_pentagram_2 = tn.replicate_nodes([state_open_pentagram], conjugate=False)[0]

In [31]:
for i in range(2):
    n.append(tn.Node(iota, name='N'+str(i)))

state_open_pentagram[-4]^n[0][1]
state_open_pentagram[-3]^n[0][2]
state_open_pentagram_2[-4]^n[0][3]
state_open_pentagram_2[-3]^n[0][4]

state_open_pentagram[-2]^n[1][1]
state_open_pentagram[-1]^n[1][2]
state_open_pentagram_2[-2]^n[1][3]
state_open_pentagram_2[-1]^n[1][4]

state_dekagram = tn.contractors.auto([state_open_pentagram, state_open_pentagram_2]+n, output_edge_order=
                                                                                        state_open_pentagram.edges[:4]+
                                                                                        [n[0].edges[0],n[1].edges[0]]+
                                                                                        state_open_pentagram_2.edges[:4])

ValueError: output edges are not equal to the remaining non-contracted edges of the final node.

In [16]:
state_dekagram = norm(state_dekagram)

In [18]:
state_dekagram.tensor.reshape(2**10)[:2**9]

array([ 3.04933923e-01,  0.00000000e+00, -1.66538942e-35,  0.00000000e+00,
        0.00000000e+00,  5.32567536e-35,  0.00000000e+00, -1.76053680e-01,
        0.00000000e+00,  1.55096743e-35,  0.00000000e+00, -1.76053680e-01,
        3.04933920e-01,  0.00000000e+00,  0.00000000e+00,  2.03289278e-01,
        2.60101853e-18,  0.00000000e+00,  6.63015106e-35,  0.00000000e+00,
        0.00000000e+00,  3.33837792e-35,  0.00000000e+00, -1.50169873e-18,
        0.00000000e+00, -3.40857385e-35,  0.00000000e+00, -1.50169873e-18,
        2.60101851e-18,  0.00000000e+00, -9.00401985e-35,  1.73401232e-18,
        2.60101853e-18, -2.43928374e-18, -9.97135162e-52, -1.04511960e-37,
        1.18754182e-35, -2.48113275e-52, -1.08494718e-18,  1.78706391e-18,
        1.18754182e-35,  3.78853292e-53, -1.08494718e-18, -1.13465362e-18,
       -5.67116760e-19,  1.21520400e-19,  0.00000000e+00,  1.31043303e-18,
        2.21861095e-35,  7.17313066e-36, -3.42471257e-52,  1.28853103e-53,
        2.54013518e-52, -

# Hexagram

In [32]:
state_open_node2 = tn.replicate_nodes([state_open_node], conjugate=False)[0]

In [33]:
links = [tn.Node(link) for i in range(4)]

n=[]
for i in range(4):
    n.append(tn.Node(iota, name='N'+str(i)))

state_open_node[1]^n[0][1]
state_open_node[2]^n[1][1]
state_open_node[3]^n[2][1]
state_open_node[4]^n[3][1]

state_open_node2[1]^n[1][2]
state_open_node2[2]^n[0][2]
state_open_node2[3]^n[3][2]
state_open_node2[4]^n[2][2]

links[0][0]^n[0][3]
links[0][1]^n[1][3]

links[1][0]^n[2][3]
links[1][1]^n[3][3]

links[2][0]^n[0][4]
links[2][1]^n[3][4]

links[3][0]^n[1][4]
links[3][1]^n[2][4]

state = tn.contractors.auto([state_open_node,state_open_node2]+n+links, output_edge_order=[state_open_node[0],n[0][0],n[1][0],state_open_node2[0],n[2][0],n[3][0]])
state_hexagram = norm(state)

In [34]:
state_hexagram.tensor.reshape(2**6)

array([-2.63342585e-01,  8.39870513e-19, -2.65783072e-18, -2.63342583e-01,
        7.36294449e-20, -3.79161902e-19,  3.03525855e-18,  2.07482540e-18,
        7.36294449e-20, -3.79161902e-19,  1.20795231e-18, -2.07482540e-18,
       -1.20795231e-18,  1.52040910e-01,  1.52040910e-01,  1.75561720e-01,
       -8.39870513e-19, -4.22955897e-19, -4.38449343e-20,  3.64837664e-18,
        3.79161902e-19,  1.52040910e-01,  1.52040910e-01,  1.75561720e-01,
       -2.63342583e-01, -6.23585828e-21, -2.74545929e-18, -2.63342580e-01,
        2.74545929e-18,  1.75561720e-01,  1.75561720e-01,  2.02721212e-01,
        2.65783072e-18, -4.38449343e-20,  3.03525855e-18, -2.74545929e-18,
       -2.63342583e-01,  1.80455478e-18, -1.18102995e-19,  8.77808601e-02,
       -1.20795231e-18,  1.52040910e-01,  1.52040910e-01, -1.75561720e-01,
        4.96550510e-19,  1.75561720e-01, -1.75561720e-01,  1.59428070e-18,
        4.38449343e-20,  1.52040910e-01,  1.52040910e-01, -1.75561720e-01,
        3.67736397e-18, -